In [310]:
import pandas as pd
import statsmodels.api as sm
from scipy import stats
from sklearn import linear_model
import numpy as np
import math

In [311]:
montana = pd.read_csv("MT-clean.csv")

/Users/BhavikaJalli/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [312]:
montana.head()

,id,state,stop_date,stop_time,location_raw,county_name,county_fips,fine_grained_location,police_department,driver_gender,...,lon,ethnicity,city,out_of_state,vehicle_year,vehicle_make,vehicle_model,vehicle_style,search_reason,stop_outcome_raw
0,MT-2009-00001,MT,2009-01-01,02:10,CASCADE,Cascade County,30013.0,US 89 N MM10 (SB),NaN,F,...,-111.802932,N,NaN,False,1994,FORD,EXPLORER,SPORT UTILITY,NaN,"TRAFFIC CITATION,WARNING"
1,MT-2009-00002,MT,2009-01-02,11:34,MISSOULA,Missoula County,30063.0,HWY 93 SO AND ANNS LANE S/B,NaN,M,...,-114.081142,N,NaN,False,1996,GMC,TK,TRUCK,NaN,"INFFRACTION ARREST,WARNING"
2,MT-2009-00003,MT,2009-01-03,11:36,MISSOULA,Missoula County,30063.0,P007 HWY 93 MM 77 N/B,NaN,M,...,-114.073505,N,NaN,False,1999,GMC,YUKON,SPORT UTILITY,NaN,INFFRACTION ARREST
3,MT-2009-00004,MT,2009-01-04,10:33,MISSOULA,Missoula County,30063.0,P007 HWY 93 MM 81 S/B,NaN,F,...,-114.079027,NaN,NaN,False,2002,HOND,CR-V,SPORT UTILITY,NaN,INFFRACTION ARREST
4,MT-2009-00005,MT,2009-01-04,10:46,MISSOULA,Missoula County,30063.0,P007 HWY 93 MM 81 N/B,NaN,M,...,-114.079150,NaN,NaN,False,1992,TOYT,TERCEL,SEDAN,NaN,INFFRACTION ARREST


In [313]:
#What proportion of traffic stops in Montana involved male drivers? 
#In other words, divide the number of traffic stops involving male drivers by the total number of stops.
total_stops = len(montana.index)
male_drivers = len(montana.loc[(montana["driver_gender"]=="M"),["driver_gender"]])
Per_maleDrivers = male_drivers/total_stops
print(Per_maleDrivers)
total_stops

0.6749749732765495


825118

In [156]:
#How many more times likely are you to be arrested in Montana during a traffic stop if you have out of state plates?
montana = montana.dropna(subset = ['out_of_state'])
out_state_arrested = len(montana.loc[(montana["out_of_state"] == True) & (montana["is_arrested"] == True)])
out_state = len(montana.loc[(montana["out_of_state"] == True)])
in_state = len(montana.loc[(montana["out_of_state"] == False)])
in_state_arrested = len(montana.loc[(montana["out_of_state"] == False) & (montana["is_arrested"] == True)])
prob_arrested_outState = out_state_arrested/out_state
prob_arrested_inState = in_state_arrested/in_state
#Since the probabilities are very small
arrests_outofState_likely = prob_arrested_outState/prob_arrested_inState
arrests_outofState_likely 

1.2095129351452942

In [232]:
#Perform a (χ2) test to determine whether the proportions of arrests in these two populations are equal. 
#What is the value of the test statistic?
#Using scipy.stats.chisquare(f_obs, f_exp=None, ddof=0, axis=0)
#The f_obs are the observed values of the proportions of arrests in each populations
f_obs = np.array([prob_arrested_outState,prob_arrested_inState])
#f_exp = None as we want to know if the proportions of arrests are equal
stats.chisquare(f_obs, f_exp=None, ddof=0, axis=0)
# montana.groupby(["out_of_state", "is_arrested"]).size()

Power_divergenceResult(statistic=0.00039264490379193408, pvalue=0.98419073700718518)

In [158]:
#What proportion of traffic stops in Montana resulted in speeding violations? 
#In other words, find the number of violations that include "Speeding" in the violation description and 
#divide that number by the total number of stops (or rows in the Montana dataset).
speeding_violations = len(montana.loc[(montana["violation"] =="Speeding")])
per_stops_speeding = speeding_violations/total_stops
per_stops_speeding

0.40734779752714156

In [159]:
vermont = pd.read_csv("VT-clean.csv")
vermont.head()

/Users/BhavikaJalli/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,state,stop_date,stop_time,location_raw,county_name,county_fips,fine_grained_location,police_department,driver_gender,...,driver_race,violation_raw,violation,search_conducted,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,officer_id
0,VT-2010-00001,VT,2010-07-01,00:10,East Montpelier,Washington County,50023.0,COUNTY RD,MIDDLESEX VSP,M,...,White,Moving Violation,Moving violation,False,No Search Conducted,NaN,False,Citation,False,-1.562157e+09
1,VT-2010-00002,VT,2010-07-01,00:10,NaN,NaN,NaN,COUNTY RD; Fitch Road,MIDDLESEX VSP,F,...,White,Externally Generated Stop,Other,False,No Search Conducted,NaN,False,Arrest for Violation,True,-1.562157e+09
2,VT-2010-00003,VT,2010-07-01,00:10,NaN,NaN,NaN,COUNTY RD; Fitch Road,MIDDLESEX VSP,F,...,White,Externally Generated Stop,Other,False,No Search Conducted,NaN,False,Arrest for Violation,True,-1.562157e+09
3,VT-2010-00004,VT,2010-07-01,00:11,Whiting,Addison County,50001.0,N MAIN ST,NEW HAVEN VSP,F,...,White,Moving Violation,Moving violation,False,No Search Conducted,NaN,False,Arrest for Violation,True,-3.126844e+08
4,VT-2010-00005,VT,2010-07-01,00:35,Hardwick,Caledonia County,50005.0,i91 nb mm 62,ROYALTON VSP,M,...,White,Moving Violation,Moving violation,False,No Search Conducted,NaN,False,Written Warning,False,9.225661e+08


In [160]:
#How much more likely does a traffic stop in Montana result in a DUI than a traffic stop in Vermont? 
#To compute the proportion of traffic stops that result in a DUI, divide the number of stops with "DUI" 
#in the violation description by the total number of stops.
per_DUI_VT = len(vermont.loc[(vermont["violation"] =="DUI")])/len(vermont.index)
per_DUI_MT = len(montana.loc[(montana["violation"] =="DUI")])/len(montana.index)
#Since the proportions are very small
DUI_MT_likely = per_DUI_MT/per_DUI_VT 
DUI_MT_likely

0.6311164497272227

In [161]:
from io import StringIO
#What is the extrapolated, average manufacture year of vehicles involved in traffic stops in Montana in 2020? 
#To answer this question, calculate the average vehicle manufacture year for each year's traffic stops. 
#Extrapolate using a linear regression.
montana.head()
montana.columns
montana["stop_date"]
montana['stop_date_processed'] = pd.to_datetime(montana['stop_date'])
montana['stop_year']= montana['stop_date_processed'].dt.year
montana['stop_year']
montana['stop_year'].dropna()
montana['vehicle_year']
montana_year = montana[["stop_year","vehicle_year"]]
montana_year.head()

,stop_year,vehicle_year
0,2009.0,1994
1,2009.0,1996
2,2009.0,1999
3,2009.0,2002
4,2009.0,1992


In [165]:
montana_year.dtypes
montana_year = montana_year.dropna()
montana_year = montana_year[pd.to_numeric(montana_year['vehicle_year'], errors='coerce').notnull()]

montana_year['vehicle_year'] = montana_year['vehicle_year'].astype(str).astype(float)
montana_year.head()

,stop_year,vehicle_year
0,2009.0,1994.0
1,2009.0,1996.0
2,2009.0,1999.0
3,2009.0,2002.0
4,2009.0,1992.0


In [202]:
avg_mfg_year = montana_year.groupby('stop_year', as_index=False)['vehicle_year'].mean()
#montana_year.dtypes
#avg_mfg_year = avg_mfg_year.round(0).astype(int)
avg_mfg_year

,stop_year,vehicle_year
0,2009.0,2000.985764
1,2010.0,2001.529400
2,2011.0,2002.293483
3,2012.0,2003.372712
4,2013.0,2003.908419
5,2014.0,2004.490274
6,2015.0,2005.307121
7,2016.0,2005.881485


In [203]:

x = avg_mfg_year.stop_year.values
y = avg_mfg_year.vehicle_year.values
x,y
length = len(x)
x = x.reshape(length, 1)
y = y.reshape(length, 1)
x,y
regr = linear_model.LinearRegression()
model = regr.fit(x, y)
test = np.array([2020])
test = test.reshape(1, 1)
model.predict(test)


array([[ 2008.85364787]])

In [207]:
#p-value of the linear regression
X = x
y = y
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
print(est.fit().f_pvalue)

5.5252654984e-08


In [241]:
#Combining both the Vermont and Montana datasets, 
#find the hours when the most and least number of traffic stops occurred. 
#What is the difference in the total number of stops that occurred in these two hours? 
#Hours range from 00 to 23. Round stop times down to compute this difference.
#I have rounded down the hours

#Concatenating the dataframes
total_data = pd.concat([montana,vermont], axis=0, ignore_index=True)
total_data.head()

,city,contraband_found,county_fips,county_name,driver_age,driver_age_raw,driver_gender,driver_race,driver_race_raw,ethnicity,...,stop_outcome,stop_outcome_raw,stop_time,stop_year,vehicle_make,vehicle_model,vehicle_style,vehicle_year,violation,violation_raw
0,NaN,False,30013.0,Cascade County,16.0,16.0,F,White,White,N,...,Citation,"TRAFFIC CITATION,WARNING",02:10,2009.0,FORD,EXPLORER,SPORT UTILITY,1994,"Other,Paperwork,Safe movement","240 - INSURANCE,150 - HIT AND RUN,245 - OTHER ..."
1,NaN,False,30063.0,Missoula County,19.0,19.0,M,White,White,N,...,Arrest,"INFFRACTION ARREST,WARNING",11:34,2009.0,GMC,TK,TRUCK,1996,"Other (non-mapped),Seat belt","EXPIRED TAG ( 4 MONTHS OR LESS ),SEATBELT ( DR..."
2,NaN,False,30063.0,Missoula County,17.0,17.0,M,White,White,N,...,Arrest,INFFRACTION ARREST,11:36,2009.0,GMC,YUKON,SPORT UTILITY,1999,Speeding,SPEED
3,NaN,False,30063.0,Missoula County,17.0,17.0,F,NaN,NaN,NaN,...,Arrest,INFFRACTION ARREST,10:33,2009.0,HOND,CR-V,SPORT UTILITY,2002,Speeding,SPEED
4,NaN,False,30063.0,Missoula County,31.0,31.0,M,NaN,NaN,NaN,...,Arrest,INFFRACTION ARREST,10:46,2009.0,TOYT,TERCEL,SEDAN,1992,"Seat belt,Speeding","SPEED,SEATBELT ( DRIVER )"


In [245]:
total_data['stop_time_processed'] = pd.to_datetime(montana['stop_time'])
total_data['stop_hour']= total_data['stop_time_processed'].dt.hour

In [252]:
total_data.groupby(["stop_hour"]).size().sort_values()

stop_hour
4.0       227
3.0       669
5.0      1085
6.0      5454
2.0      6148
1.0     10321
0.0     14823
23.0    25546
22.0    31654
7.0     32796
21.0    34099
12.0    34482
20.0    36081
11.0    39924
19.0    41818
9.0     45166
13.0    45811
8.0     47152
10.0    47252
18.0    55755
17.0    57219
14.0    64230
16.0    67473
15.0    75223
dtype: int64

In [307]:
#We can use the traffic stop locations to estimate the areas of the counties in Montana. 
#Represent each county as an ellipse with semi-axes given by a single standard deviation of 
#the longitude and latitude of stops within that county. 
#What is the area, in square kilometers, of the largest county measured in this manner? 
#Please ignore unrealistic latitude and longitude coordinates.
#SInce Montana is in the Western Hemisphere, all the positive values of the longitude can be disgarded
montana= montana[montana['lon'] < 0]
mean_latitude = np.mean(montana['lat'])
long_distance = math.radians(mean_latitude) * 111
long_distance

90.82154979000504

In [309]:
import math
montana_axes = montana.groupby(["county_name"]).aggregate({'lat': 'std','lon': 'std'})
montana_axes['lat_km'] = montana_axes['lat']*111
montana_axes['lon_km'] = montana_axes['lon']*long_distance
montana_axes["county_area"] = math.pi * montana_axes["lat_km"] * montana_axes["lon_km"]
montana_axes["county_area"].sort_values()

county_name
Wibaux County               180.832265
Treasure County             199.438766
Silver Bow County           211.846079
Golden Valley County        230.922010
Liberty County              292.803132
Sweet Grass County          300.361981
Stillwater County           304.816566
Ravalli County              341.177591
Blaine County               345.645594
Granite County              355.965899
Prairie County              356.994514
Musselshell County          393.545244
Flathead County             404.011357
Wheatland County            430.818338
Fallon County               433.242103
Hill County                 473.922594
Daniels County              495.261374
Petroleum County            505.460523
Meagher County              528.324748
Sheridan County             534.268324
Missoula County             539.388285
Yellowstone County          555.840138
Pondera County              564.870338
Dawson County               582.243809
Custer County               628.742166
Broadwater Co